In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_validate
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
DF_train = pd.read_csv('/kaggle/input/water-pollution/Train.csv', delimiter = ',')
DF_test = pd.read_csv('/kaggle/input/water-pollution/Test.csv', delimiter = ',')
Target = pd.read_csv('/kaggle/input/water-pollution/Target.csv', delimiter = ',')
Submission = pd.read_csv('/kaggle/input/water-pollution/Submission.csv', delimiter = ',')

In [ ]:
len(set(DF_train['id'].values).intersection(set(DF_test['id'].values)))/len(set(DF_test['id'].values))

In [ ]:
# ID's presented in train and in test

df_id = set(DF_train['id'].values).intersection(set(DF_test['id'].values))
DF_test[DF_test['id'].isin(df_id)]['id']
#DF_test = DF_test[DF_test['id'].isin(df_id)]

In [ ]:
df = pd.concat([DF_train, DF_test])

In [ ]:
df = df.drop(columns = ['Unnamed: 0'])

In [ ]:
cat_columns = []
num_columns = []

for column_name in df.columns:
    if (df[column_name].dtypes == object):
        cat_columns +=[column_name]
    else:
        num_columns +=[column_name]

print('categorical columns:\t ',cat_columns, '\n len = ',len(cat_columns))

print('numerical columns:\t ',  num_columns, '\n len = ',len(num_columns))

In [ ]:
cat_columns = ['code', 'period', 'id', 'Country', 'year']
num_columns = ['Unnamed: 0', 'tourists', 'venue', 'rate', 'food', 'glass', 'metal', 'other', 'paper', 'plastic', 'leather', 'green_waste', 'waste_recycling'] 


In [ ]:
df_ohe = pd.get_dummies(df[cat_columns])

In [ ]:
df_all = pd.concat([df_ohe, df[num_columns]], axis=1)

In [ ]:
# Creating StandardScaler Object
scaler = StandardScaler()
df_scale = pd.DataFrame(scaler.fit_transform(df_all[num_columns]))
df_all = pd.concat([df_ohe.reset_index(drop=True), df_scale.reset_index(drop=True)], axis=1)

In [ ]:
len_test = len(DF_test)
train = df_all.iloc[:-len_test]
test = df_all.iloc[-len_test:]
print(DF_train.shape[0], train.shape[0], DF_test.shape[0], test.shape[0])

In [ ]:
#target_log = np.log(Target['polution'])
#target_log
target_orig = Target['polution']
len(pd.DataFrame(train))

In [ ]:
X, y = pd.DataFrame(train).values, target_orig.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape[0], X_test.shape[0], y_train.shape[0], y_test.shape[0]

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
### define model
model = Ridge()

alphas = np.array([1,0.1,0.01,0.001,0.0001,0.000001, 0.0000001])

# define parameters
param = {
    'alpha':alphas,
    'positive':[True, False],
    'copy_X':[True,False],
    'fit_intercept':[True,False]
    
}
grid = GridSearchCV(model, param, scoring='r2', cv=2, n_jobs=-1)
grid.fit(X_train, y_train)
predictions = grid.predict(X_test)
print('R2 : %.4f' %  r2_score(y_test,predictions))
print('R2 : %.4f' % grid.best_score_)

In [ ]:
model_ElasticCV = ElasticNetCV(n_alphas=1, fit_intercept=True, max_iter=100000, l1_ratio=0.9, eps=0.0001, cv=5)

#model_ElasticCV.fit(X_train, y_train)

#y_predict_ElasticCV = model_ElasticCV.predict(X_test)

#print('R2 : %.4f' %  r2_score(y_test,y_predict_ElasticCV))

In [ ]:
model_Elastic = ElasticNet(alpha=0.00005, fit_intercept=True, max_iter=100000, l1_ratio=0.07)

model_Elastic.fit(X_train, y_train)

y_predict_Elastic = model_Elastic.predict(X_test)

print('R2 : %.4f' %  r2_score(y_test,y_predict_Elastic))

In [ ]:

model = Ridge(alpha = 0.1, fit_intercept=True, max_iter=10000)

model.fit(X_train, y_train)

y_predict=model.predict(X_test)

print('Ошибка на тестовых данных')
print('MSE: %.1f' % mse(y_test,y_predict))
print('RMSE: %.1f' % mse(y_test,y_predict,squared=False))
print('R2 : %.4f' %  r2_score(y_test,y_predict))

In [ ]:
model_L = Lasso(alpha=0.00001, fit_intercept=True)

model_L.fit(X_train, y_train)
y_predict=model_L.predict(X_test)

print('R2 : %.4f' %  r2_score(y_test,y_predict))

In [ ]:
def plotGraph(y_test,y_predict,regressorName):
    if max(y_test) >= max(y_predict):
        my_range = int(max(y_test))
    else:
        my_range = int(max(y_predict))
    plt.scatter(range(len(y_test)), y_test, color='blue')
    plt.scatter(range(len(y_predict)), y_predict, color='red')
    plt.title(regressorName)
    plt.show()
    return


plotGraph(y_test, y_predict_Elastic, "RED - predicted. BLUE - original")
plotGraph(y_test, y_predict, "RED - predicted. BLUE - original")

In [ ]:
def cross_val(model, X_train, y_train):
  scoring = {'R2': 'r2',
            '-MSE': 'neg_mean_squared_error',
            '-MAE': 'neg_mean_absolute_error',
            'Max': 'max_error'}


  scores = cross_validate(model, X_train, y_train,
                        scoring=scoring, cv=ShuffleSplit(n_splits=5, random_state=42) )

  print('Результаты Кросс-валидации')
  DF_cv_linreg = pd.DataFrame(scores)
  display(DF_cv_linreg)
  print('\n')
  print(DF_cv_linreg.mean()[2:])
  print('\n')

cross_val(model_Elastic, X_train, y_train)

In [ ]:
model = Ridge(alpha=0.0001, max_iter=10000)

model.fit(X_train, y_train)

y_predict = model_Elastic.predict(test)

#y_predict = np.exp(y_predict)
Submission['polution'] = y_predict
Submission

In [ ]:
! rm /kaggle/working/water-pollution/Submission.csv
! mkdir /kaggle/working/water-pollution/
Submission['polution'][27] = 2.0

In [ ]:
Submission.to_csv('/kaggle/working/water-pollution/Submission.csv',index=False)

In [ ]:
! mkdir ~/.kaggle
! touch ~/.kaggle/kaggle.json
! echo '{"username":"hydramst","key": "" }' > ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
cat ~/.kaggle/kaggle.json

In [ ]:
#! kaggle competitions submit -f /kaggle/working/water-pollution/Submission.csv -m "Message" sf-matml-2022-regression